In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from uuid import uuid4

# import the .env file
from dotenv import load_dotenv
load_dotenv()

False

In [2]:
# configuration
DATA_PATH = r"data"
CHROMA_PATH = r"chroma_db"

In [3]:
from langchain_ollama import OllamaEmbeddings


embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [4]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory=CHROMA_PATH,
)

In [5]:
loader = PyPDFDirectoryLoader(DATA_PATH)

In [6]:
raw_documents = loader.load()

In [7]:
# splitting the document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

# creating the chunks
chunks = text_splitter.split_documents(raw_documents)

In [8]:
uuids = [str(uuid4()) for _ in range(len(chunks))]

# adding chunks to vector store
vector_store.add_documents(documents=chunks, ids=uuids)

['f4154142-791f-41fd-8095-1a8816bcf559',
 '4092557b-8d17-4c78-967d-bea5eba05a25',
 '3832e103-2537-4202-9429-e666c676d241',
 '7dc05bbd-25a2-4c52-b9c3-e31b131826af',
 'bd54f8b3-61c3-4a4f-a0d1-91dfc333bf50',
 '5390a0e7-fc85-4a62-91b3-a1b59785a3ef',
 '270fbdf7-a9a3-48a3-a80a-e5ce095dcbaa',
 'd8b82a80-e7cf-4b59-aaa9-db3b217d4862',
 '7b4632cc-fb59-41b9-a187-4c1709d2398c',
 '43be785d-a498-43c2-b2cb-524e782aa84d',
 '784d43d0-8825-499e-9de4-41598d3a31e6',
 '176ad524-2b82-4b9e-bfd5-e835ac615389',
 '60314725-a8d2-4111-8cc2-cdec39f0038b',
 'a2cea444-8776-4a36-b9e3-6dd82fd75b77',
 '3c16b035-d613-47f4-b863-94467f079cbd',
 'a64dd516-4fa3-45ce-b56d-174bd7660a08',
 '403e612c-779c-4967-bacb-0b8b012b8661',
 '90f81874-4be4-42f3-abf1-b037d923ba49',
 'f50dea84-b041-4458-86cd-dc55d260cbda',
 '739318af-b685-47fa-b76b-29447f40b2fb',
 '88ae637c-c5ec-466e-afa5-46384099f539',
 '0ac698de-69f7-43b9-a06a-c0308601bfaa',
 '7c3f886c-00e5-4c82-a2bd-9487c4bf37ca',
 '77d80745-ba21-4801-afe0-b5282a888da7',
 '38749e8c-a52f-

In [9]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
import gradio as gr

# import the .env file
from dotenv import load_dotenv
load_dotenv()


/home/ezzoubair/Desktop/TTX/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [ ]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage
from langchain_ollama import OllamaEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
load_dotenv()

GROQ_API_KEY = ""
llm_compression = ChatGroq(temperature=0,api_key=GROQ_API_KEY,model="llama-3.3-70b-versatile")
compressor_llm = LLMChainExtractor.from_llm(llm_compression)

In [ ]:
# Set up the vectorstore to be the retriever


num_results = 5
retriever = vector_store.as_retriever(search_kwargs={'k': num_results})



In [25]:
advanced_retriever = ContextualCompressionRetriever(
    base_compressor=compressor_llm,
    base_retriever=retriever
)

def retrieve_context(query):
    """Récupère le contexte pertinent pour la question"""
    docs = advanced_retriever.get_relevant_documents(query)
    return "\n\n".join([f"- {doc.page_content}" for doc in docs])

# --- Configuration du prompt RAG ---
rag_template = """**Rôle** : Expert in analysing repport .
**Mission** : Giving though in how to improve the repport.

**Historique** :
{history}

**Connaissances** :
{context}

**Question** :
{question}

**Consignes** :
- Répondez uniquement avec les informations disponibles
- En cas de hors-sujet : "Je n'ai pas cette information. Voulez-vous une question sur les tendances énergétiques marocaines ?"
- Ton professionnel et clair

**Format** :
Structure claire avec informations factuelles
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)


llm_response = ChatGroq(temperature=0,model="llama-3.3-70b-versatile",api_key=GROQ_API_KEY)


memory = ConversationBufferMemory(memory_key="history", return_messages=False)

def generate_response(user_message):
    """Génère la réponse de l'assistant"""

    conversation_history = memory.buffer


    context = retrieve_context(user_message)


    prompt = rag_prompt.format(
        history=conversation_history,
        context=context,
        question=user_message
    )


    response = llm_response([HumanMessage(content=prompt)]).content


    memory.save_context({"input": user_message}, {"output": response})

    return response

In [26]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## Djnago ")
    chatbot = gr.Chatbot()
    state = gr.State([])

    with gr.Row():
        txt_input = gr.Textbox(
            show_label=False,
            placeholder="Posez votre question ici...",
            container=False
        )
        send_btn = gr.Button("Envoyer")

    def user_interaction(user_message, history):
        response = generate_response(user_message)
        history.append((user_message, response))
        return "", history

    txt_input.submit(user_interaction, [txt_input, state], [txt_input, chatbot])
    send_btn.click(user_interaction, [txt_input, state], [txt_input, chatbot])

demo.launch()

/tmp/ipykernel_44928/1037708201.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


/tmp/ipykernel_44928/3279438843.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = advanced_retriever.get_relevant_documents(query)
/tmp/ipykernel_44928/3279438843.py:57: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_response([HumanMessage(content=prompt)]).content
